### Team 39 Contributors:
* 20241457 Onni Mansikkamäki
* 20240506 Jan-Louis Schneider
* 20240945 Emir Kamiloglu
* 20240941 Tomás Figueiredo
* 20240661 Rita Santos

In [2]:
%pip install summarytools
%pip install ipywidgets
%pip install annoy
%pip install tensorflow
%pip install imbalanced-learn

from annoy import AnnoyIndex
import phik
import warnings
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import StandardScaler
print("StandardScaler imported")
from sklearn.neural_network import MLPClassifier
import seaborn as sns
from summarytools import dfSummary
from math import ceil
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.decomposition import PCA
import scipy.stats as stats 

import joblib

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
StandardScaler imported


In [3]:
#Import the data

with open("claim_injury_type_mapping.json", "r") as file:
    claim_injury_type_mapping = json.load(file)

train_data_standardized = pd.read_csv("train_data_processed_standardized.csv")
train_data_non_standardized = pd.read_csv("train_data_processed_non_standardized.csv")

test_data_standardized = pd.read_csv("test_data_processed_standardized.csv")
test_data_non_standardized = pd.read_csv("test_data_processed_non_standardized.csv")

validation_data_standardized = pd.read_csv("validation_data_processed_standardized.csv")
validation_data_non_standardized = pd.read_csv("validation_data_processed_non_standardized.csv")


In [6]:
# Check the data
print(train_data_standardized.columns)
train_data_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'Claim Injury Type', 'District Name', 'Gender',
       'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,Claim Injury Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,-0.658501,-0.763726,-0.609139,0,0.668627,-0.581041,1,3,1,1,0.839945,42,25,49,55,-1.501980,-0.268603
1,0.923888,-0.763726,0.627758,0,-0.496233,0.618894,1,1,6,2,-0.418975,33,98,59,34,-1.002320,1.136203
2,0.027909,-0.689781,0.138087,0,-0.480410,0.151551,2,2,6,1,-0.418975,62,19,43,37,1.495978,-0.364385
3,1.494884,-1.355279,1.664335,0,-0.480410,1.670417,1,2,3,1,-0.418975,62,45,7,12,-1.501980,-0.332457
4,-1.314540,1.676432,-1.318208,0,-0.854524,-1.294687,1,3,8,1,-0.418975,44,56,49,38,-1.002320,-0.194105


In [7]:
print(train_data_non_standardized.columns)
train_data_non_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'Claim Injury Type', 'District Name', 'Gender',
       'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,Claim Injury Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,1048,32,1039,0,1146.82,1042,1,3,1,1,2.0,42,25,49,55,0.0,9
1,1569,32,1428,0,443.79,1422,1,1,6,2,0.0,33,98,59,34,1.0,141
2,1274,33,1274,0,453.34,1274,2,2,6,1,0.0,62,19,43,37,6.0,0
3,1757,24,1754,0,453.34,1755,1,2,3,1,0.0,62,45,7,12,0.0,3
4,832,65,816,0,227.55,816,1,3,8,1,0.0,44,56,49,38,1.0,16


In [8]:
print(test_data_standardized.columns)
test_data_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'District Name', 'Gender', 'IME-4 Count',
       'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,-1.685080,-1.725000,-1.683872,0,-0.178122,-1.657825,1,5,1,-0.418975,48,31,10,54,-1.002320,-0.268603
1,-1.581815,-1.725000,-1.683872,0,-0.793964,-1.657825,1,5,2,-0.418975,45,75,10,10,-1.002320,0.093241
2,-1.691155,1.232767,-1.683872,0,-1.231555,-1.651510,1,5,2,-0.418975,56,68,49,62,-1.501980,-0.289887
3,-1.697229,0.936991,-1.683872,0,-1.231555,-1.657825,1,5,2,-0.418975,48,25,10,53,1.495978,-0.311172
4,-1.672931,-1.281335,-1.683872,0,-1.231555,-1.651510,1,5,1,-0.418975,55,79,40,37,0.996319,-0.226033


In [9]:
print(test_data_non_standardized.columns)
test_data_non_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'District Name', 'Gender', 'IME-4 Count',
       'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,710,19,701,0,635.78,701,1,5,1,0.0,48,31,10,54,1.0,9
1,744,19,701,0,264.10,701,1,5,2,0.0,45,75,10,10,1.0,43
2,708,59,701,0,0.00,703,1,5,2,0.0,56,68,49,62,0.0,7
3,706,55,701,0,0.00,701,1,5,2,0.0,48,25,10,53,6.0,5
4,714,25,701,0,0.00,703,1,5,1,0.0,55,79,40,37,5.0,13


In [10]:
print(validation_data_standardized.columns)
validation_data_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'Claim Injury Type', 'District Name', 'Gender',
       'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,Claim Injury Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,0.392375,0.863046,0.379743,0,-0.758655,0.214706,3,2,5,2,-0.418975,61,74,10,35,0.996319,0.103884
1,0.434896,-1.429223,0.519649,0,-0.480410,0.530478,4,2,1,2,-0.418975,62,74,52,53,0.496659,-0.215390
2,1.595112,-0.467949,1.740647,0,1.164641,1.743045,1,4,3,2,-0.418975,62,50,49,38,-0.003001,-0.236675
3,-1.153567,0.789102,-1.127427,0,-0.793964,-1.095750,2,2,2,1,-0.418975,45,60,52,35,-0.003001,-0.268603
4,0.577645,1.010935,0.608680,0,-0.480410,0.618894,4,2,5,2,-0.418975,62,19,43,36,-0.502661,-0.013183


In [11]:
print(validation_data_non_standardized.columns)
validation_data_non_standardized.head()

Index(['Accident Date', 'Age at Injury', 'Assembly Date',
       'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date',
       'Carrier Type', 'Claim Injury Type', 'District Name', 'Gender',
       'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code',
       'Number of Dependents', 'Days Until Injury Reported'],
      dtype='object')


,Accident Date,Age at Injury,Assembly Date,Attorney/Representative,Average Weekly Wage,C-2 Date,Carrier Type,Claim Injury Type,District Name,Gender,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Days Until Injury Reported
0,1394,54,1350,0,285.41,1294,3,2,5,2,0.0,61,74,10,35,5.0,44
1,1408,23,1394,0,453.34,1394,4,2,1,2,0.0,62,74,52,53,4.0,14
2,1790,36,1778,0,1446.18,1778,1,4,3,2,0.0,62,50,49,38,3.0,12
3,885,53,876,0,264.10,879,2,2,2,1,0.0,45,60,52,35,3.0,9
4,1455,56,1422,0,453.34,1422,4,2,5,2,0.0,62,19,43,36,2.0,33


 ### KNN Classifier with PCA to reduce dimensionality of data.

 * Normal KNN, it took so much time that for training, data was sampled from training data in the beginning. Later more data was added to training

 * Approximation of NN calculation was used so that the calculation would be faster

     * Annoy library (Approximate Nearest Neighbors Oh Yeah) was used to get the approximation of the NN

         * Annoy algorithm builds multiple trees for fast approximate neighbor search. A higher number of trees increases accuracy but slows down building of the tree.



In [ ]:
# %%
# #KNN with PCA, data sampling Annoy optimization

# train_data_knn = train_data.copy()
# test_data_knn = test_data.copy()

# # Define the independant variables (as data) and the target variable
# data_knn = train_data_knn.drop(["Claim Injury Type"], axis=1)
# target_knn = train_data_knn["Claim Injury Type"]

# # Split data to X_train, X_test, y_train and y_test
# X_train_knn, X_val_knn, y_train_knn, y_val_knn = train_test_split(data_knn, 
#                                                   target_knn, 
#                                                   test_size = 0.2, 
#                                                   random_state=5, 
#                                                   stratify = target_knn)


# # Sample and reduce dimensionality of the data
# sample_size = 400000
# X_train_knn_sample, _, y_train_knn_sample, _ = train_test_split(X_train_knn, y_train_knn, train_size=sample_size, random_state=42)
# # Determine the number of PC's based on available features or samples
# n_components = min(15, X_train_knn_sample.shape[1])
# pca = PCA(n_components=n_components)
# X_train_knn_reduced = pca.fit_transform(X_train_knn_sample)
# X_val_knn_reduced = pca.transform(X_val_knn)

# # Build Annoy index with reduced data
# index = AnnoyIndex(X_train_knn_reduced.shape[1], 'euclidean')
# for i, vector in enumerate(X_train_knn_reduced):
#     index.add_item(i, vector)
# index.build(10)  # Number of trees in Annoy; higher means better accuracy but slower build


# # Convert y_train_knn_sample to a NumPy array for positional indexing
# y_train_knn_sample = y_train_knn_sample.to_numpy()

# # Function to get predictions using Annoy 
# '''
# Retrieves the indices of the k nearest neighbors for the vector x in X_val. These are approximate neighbors based on the trees built by Annoy.
# '''

# def get_approximate_knn_predictions(X_val, index, y_train, k):
#     predictions = []
#     for x in X_val:
#         nearest_indices = index.get_nns_by_vector(x, k)
#         nearest_labels = [y_train[idx] for idx in nearest_indices]
#         predictions.append(max(set(nearest_labels), key=nearest_labels.count))
#     return predictions

# # Evaluate each range of k values
# k_ranges = {
#     "2-10": range(2, 11),
#     "11-20": range(11, 21),
#     "21-30": range(21, 31)
#     # "100-110": range(100, 111)
# }

# best_k_values = {}
# best_accuracy_scores = {}
# best_f1_scores = {}
# best_predictions = {}

# for label, k_range in k_ranges.items():
#     accuracy_scores = []
#     f1_scores = []
#     high_accuracy = 0
#     best_k = 0
#     best_preds = None
#     best_f1 = 0  # Track F1 score for the best k based on accuracy

#     print(f"Evaluating k values in range: {label}")
    
#     for k in k_range:
#         # Get approximate KNN predictions
#         predictions = get_approximate_knn_predictions(X_val_knn_reduced, index, y_train_knn_sample, k)
        
#         # Calculate accuracy and F1 score
#         accuracy = accuracy_score(y_val_knn, predictions)
#         f1 = f1_score(y_val_knn, predictions, average='weighted')
        
#         accuracy_scores.append(accuracy)
#         f1_scores.append(f1)

#         # Update best k if this model has a higher accuracy
#         if accuracy > high_accuracy:
#             high_accuracy = accuracy
#             best_k = k
#             best_preds = predictions
#             best_f1 = f1  

#     # Store results for this range
#     best_k_values[label] = best_k
#     best_accuracy_scores[label] = high_accuracy
#     best_f1_scores[label] = best_f1
#     best_predictions[label] = best_preds

#     # Plot Accuracy and F1 Score for the current k range
#     plt.figure()
#     plt.plot(list(k_range), accuracy_scores, marker='o', label='Accuracy')
#     plt.plot(list(k_range), f1_scores, marker='x', label='F1 Score')
#     plt.xlabel('Number of Neighbors (k)')
#     plt.ylabel('Score')
#     plt.title(f'Accuracy and F1 Score for k values in Range {label}')
#     plt.legend()
#     plt.show()

#     # Output the best result for the range
#     print(f"Best k in range {label}: {best_k}")
#     print(f"Best Accuracy in range {label}: {high_accuracy}")
#     print(f"F1 Score for best k ({best_k}) in range {label}: {best_f1}")




 # We saw that with k over 21, we are not gaining accuracy or f1 score anymore. Let's now try more interesting k-values under 21 and also see the training data accuracy to see if we are overfitting.



 # We found that training and evaluation accuracy are getting closer to each other without losing val accuracy with under k<20. Let's try also k values over 20 and see if we can reduce overfitting without losing too much f1 score and validation accuracy.

In [ ]:
# %%
# train_data_knn = train_data.copy()
# test_data_knn = test_data.copy()

# # Define the independant variables (as data) and the target variable
# data_knn = train_data_knn.drop(["Claim Injury Type"], axis=1)
# target_knn = train_data_knn["Claim Injury Type"]

# # Split data to X_train, X_test, y_train and y_test

# X_train_knn, X_val_knn, y_train_knn, y_val_knn = train_test_split(data_knn, 
#                                                   target_knn, 
#                                                   test_size = 0.2, 
#                                                   random_state=5, 
#                                                   stratify = target_knn)

# # Sample and reduce dimensionality of the data
# sample_size = 400000
# X_train_knn_sample, _, y_train_knn_sample, _ = train_test_split(X_train_knn, y_train_knn, train_size=sample_size, random_state=42)
# pca = PCA(n_components=min(15, X_train_knn_sample.shape[1]))  # Adjusting PCA components as needed
# X_train_knn_reduced = pca.fit_transform(X_train_knn_sample)
# X_val_knn_reduced = pca.transform(X_val_knn)

# # Convert y_train_knn_sample to a NumPy array for reliable indexing
# y_train_knn_sample = y_train_knn_sample.to_numpy()

# #  Build Annoy index with reduced data
# index = AnnoyIndex(X_train_knn_reduced.shape[1], 'euclidean')
# for i, vector in enumerate(X_train_knn_reduced):
#     index.add_item(i, vector)
# index.build(10)  

# # Function to get predictions using Annoy
# def get_approximate_knn_predictions(X_val, index, y_train, k):
#     predictions = []
#     for x in X_val:
#         nearest_indices = index.get_nns_by_vector(x, k)
#         nearest_labels = [y_train[idx] for idx in nearest_indices]
#         predictions.append(max(set(nearest_labels), key=nearest_labels.count))
#     return predictions

# # Evaluate each range of k values

# k_ranges = {
#     "2-10": range(2, 11),
#     "11-20": range(11, 21),
#     "21-40": range(21, 41),
# }

# best_k_values = {}
# best_accuracy_scores = {}
# best_f1_scores = {}
# best_predictions = {}

# for label, k_range in k_ranges.items():
#     accuracy_scores_train = []
#     accuracy_scores_val = []
#     f1_scores = []
#     high_accuracy = 0
#     best_k = 0
#     best_preds = None
#     best_f1 = 0  # Track F1 score for the best k based on validation accuracy

#     print(f"Evaluating k values in range: {label}")
    
#     for k in k_range:
#         # Get predictions and accuracy for the validation data
#         predictions_val = get_approximate_knn_predictions(X_val_knn_reduced, index, y_train_knn_sample, k)
#         accuracy_val = accuracy_score(y_val_knn, predictions_val)
#         f1_val = f1_score(y_val_knn, predictions_val, average='weighted')

#         # Get predictions and accuracy for the training data
#         predictions_train = get_approximate_knn_predictions(X_train_knn_reduced, index, y_train_knn_sample, k)
#         accuracy_train = accuracy_score(y_train_knn_sample, predictions_train)

#         # Store training and validation accuracy and F1 score
#         accuracy_scores_train.append(accuracy_train)
#         accuracy_scores_val.append(accuracy_val)
#         f1_scores.append(f1_val)

#         # Update best k if this model has a higher validation accuracy
#         if accuracy_val > high_accuracy:
#             high_accuracy = accuracy_val
#             best_k = k
#             best_preds = predictions_val
#             best_f1 = f1_val  # Store the F1 score for the best accuracy on validation data

#     # Store results for this range
#     best_k_values[label] = best_k
#     best_accuracy_scores[label] = high_accuracy
#     best_f1_scores[label] = best_f1
#     best_predictions[label] = best_preds

#     # Plot Accuracy (Train and Validation) and F1 Score for the current k range
#     plt.figure()
#     plt.plot(list(k_range), accuracy_scores_train, marker='o', label='Training Accuracy')
#     plt.plot(list(k_range), accuracy_scores_val, marker='x', label='Validation Accuracy')
#     plt.plot(list(k_range), f1_scores, marker='^', label='Validation F1 Score')
#     plt.xlabel('Number of Neighbors (k)')
#     plt.ylabel('Score')
#     plt.title(f'Accuracy and F1 Score for k values in Range {label}')
#     plt.legend()
#     plt.show()

#     # Output the best result for the range
#     print(f"Best k in range {label}: {best_k}")
#     print(f"Best Validation Accuracy in range {label}: {high_accuracy}")
#     print(f"F1 Score for best k ({best_k}) in range {label}: {best_f1}")
#     print(f"Training Accuracy for best k ({best_k}) in range {label}: {accuracy_scores_train[k_range.index(best_k)]}")




 ## Best K taking accuracy in both val and test set in account seems to be k=35. Even though training accuracy is dropping, the overfitting is reduced with higher k. Validation performance is staying quite consistent with higher k and f1 is not dropping significantly. Later in optimization we might want to oversample training data that has the minority class value of the traget variable. Now our model is probably not good in classification of those minority target variable class rows.



 ## Even though best validation accuracy is found with k=24, I prefer to have better generalizing model (less overfitting) than only 0.1% better validation accuracy. Higher k than 35 is not chosen due to efficiency reasons at this point. Maybe later with more time in our hands we could go for higher k.



 ## Even with k=35, model is slightly overfitting still.

In [ ]:
# %%
# # Transform the test data with PCA
# X_test_knn_reduced = pca.transform(test_data_knn) 

# # Make predictions on the transformed test data using the best k value
# best_k = 35 # Replace with the range key that had the best k (35)
# test_predictions = get_approximate_knn_predictions(X_test_knn_reduced, index, y_train_knn_sample, best_k)



In [ ]:
# %%

# test_predictions = [int(pred) for pred in test_predictions]

# print(test_predictions)
# print(claim_injury_type_mapping)

# # Format the predictions according to the required label format

# # # Convert numerical predictions to the required string format using label mapping
# formatted_predictions = [claim_injury_type_mapping[pred] for pred in test_predictions]

# formatted_predictions

# # # Create the submission DataFrame
# submission = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions
# })

# submission.head()
# # # Save to CSV in the required format
# submission.to_csv("model1.csv", index=False)

# print("Submission file created: model1.csv")


 ## Experiment with standardization for also categorial predictor variables (MODEL FOR BEST KAGGLE RESULT)

In [ ]:
# %%
# train_data_knn = train_data_non_standardized.copy()
# test_data_knn = test_data_non_standardized.copy()

# # Get all columns from the train data which are not the target variable
# predictor_cols = train_data_knn.columns[train_data_knn.columns != "Claim Injury Type"]

# # Initialize the StandardScaler
# scaler = StandardScaler()
# scaler.fit(train_data_knn[predictor_cols])

# # Scale the columns in both train and test data
# train_data_knn[predictor_cols] = scaler.transform(train_data_knn[predictor_cols])
# test_data_knn[predictor_cols] = scaler.transform(test_data_knn[predictor_cols])


# # Define the independant variables (as data) and the target variable
# data_knn = train_data_knn.drop(["Claim Injury Type"], axis=1)
# target_knn = train_data_knn["Claim Injury Type"]

# # Split data to X_train, X_test, y_train and y_test
# X_train_knn, X_val_knn, y_train_knn, y_val_knn = train_test_split(data_knn, 
#                                                   target_knn, 
#                                                   test_size = 0.2, 
#                                                   random_state=5, 
#                                                   stratify = target_knn)

# # Sample and reduce dimensionality of the data
# sample_size = 400000
# X_train_knn_sample, _, y_train_knn_sample, _ = train_test_split(X_train_knn, y_train_knn, train_size=sample_size, random_state=42)
# pca = PCA(n_components=min(15, X_train_knn_sample.shape[1]))  # Adjusting PCA components as needed
# X_train_knn_reduced = pca.fit_transform(X_train_knn_sample)
# X_val_knn_reduced = pca.transform(X_val_knn)

# # Convert y_train_knn_sample to a NumPy array for reliable indexing
# y_train_knn_sample = y_train_knn_sample.to_numpy()

# #  Build Annoy index with reduced data
# index = AnnoyIndex(X_train_knn_reduced.shape[1], 'euclidean')
# for i, vector in enumerate(X_train_knn_reduced):
#     index.add_item(i, vector)
# index.build(10)  

# # Function to get predictions using Annoy
# def get_approximate_knn_predictions(X_val, index, y_train, k):
#     predictions = []
#     for x in X_val:
#         nearest_indices = index.get_nns_by_vector(x, k)
#         nearest_labels = [y_train[idx] for idx in nearest_indices]
#         predictions.append(max(set(nearest_labels), key=nearest_labels.count))
#     return predictions

# # Evaluate each range of k values
# k_ranges = {
#     "30-40": range(30, 41),
# }

# best_k_values = {}
# best_accuracy_scores = {}
# best_f1_scores = {}
# best_predictions = {}

# for label, k_range in k_ranges.items():
#     accuracy_scores_train = []
#     accuracy_scores_val = []
#     f1_scores = []
#     high_accuracy = 0
#     best_k = 0
#     best_preds = None
#     best_f1 = 0  # Track F1 score for the best k based on validation accuracy

#     print(f"Evaluating k values in range: {label}")
    
#     for k in k_range:
#         # Get predictions and accuracy for the validation data
#         predictions_val = get_approximate_knn_predictions(X_val_knn_reduced, index, y_train_knn_sample, k)
#         accuracy_val = accuracy_score(y_val_knn, predictions_val)
#         f1_val = f1_score(y_val_knn, predictions_val, average='weighted')

#         # Get predictions and accuracy for the training data
#         predictions_train = get_approximate_knn_predictions(X_train_knn_reduced, index, y_train_knn_sample, k)
#         accuracy_train = accuracy_score(y_train_knn_sample, predictions_train)

#         # Store training and validation accuracy and F1 score
#         accuracy_scores_train.append(accuracy_train)
#         accuracy_scores_val.append(accuracy_val)
#         f1_scores.append(f1_val)

#         # Update best k if this model has a higher validation accuracy
#         if accuracy_val > high_accuracy:
#             high_accuracy = accuracy_val
#             best_k = k
#             best_preds = predictions_val
#             best_f1 = f1_val  # Store the F1 score for the best accuracy on validation data

#     # Store results for this range
#     best_k_values[label] = best_k
#     best_accuracy_scores[label] = high_accuracy
#     best_f1_scores[label] = best_f1
#     best_predictions[label] = best_preds

#     # Plot Accuracy (Train and Validation) and F1 Score for the current k range
#     plt.figure()
#     plt.plot(list(k_range), accuracy_scores_train, marker='o', label='Training Accuracy')
#     plt.plot(list(k_range), accuracy_scores_val, marker='x', label='Validation Accuracy')
#     plt.plot(list(k_range), f1_scores, marker='^', label='Validation F1 Score')
#     plt.xlabel('Number of Neighbors (k)')
#     plt.ylabel('Score')
#     plt.title(f'Accuracy and F1 Score for k values in Range {label}')
#     plt.legend()
#     plt.show()

#     # Output the best result for the range
#     print(f"Best k in range {label}: {best_k}")
#     print(f"Best Validation Accuracy in range {label}: {high_accuracy}")
#     print(f"F1 Score for best k ({best_k}) in range {label}: {best_f1}")
#     print(f"Training Accuracy for best k ({best_k}) in range {label}: {accuracy_scores_train[k_range.index(best_k)]}")



 ## Best k seems to be 37 in this case from range 30 to 40.



In [ ]:
# %%
# # Transform the test data with PCA
# X_test_knn_reduced = pca.transform(test_data_knn) 

# # Make predictions on the transformed test data using the best k value
# best_k = 37 # Replace with the range key that had the best k (37)
# test_predictions = get_approximate_knn_predictions(X_test_knn_reduced, index, y_train_knn_sample, best_k)

# test_predictions = [int(pred) for pred in test_predictions]

# # Format the predictions according to the required label format

# # # Convert numerical predictions to the required string format using label mapping
# formatted_predictions = [claim_injury_type_mapping[pred] for pred in test_predictions]

# formatted_predictions

# # # Create the submission DataFrame
# submission = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions
# })

# submission.head()
# # # Save to CSV in the required format
# submission.to_csv("model4.csv", index=False)

# print("Submission file created: model4.csv")


 ### Optimizations and results

 * **Increase training data points (from sample of 50 000 to all of the data)**

     * With 50 000 samples of training data

         * **We got f1 score of 0.23926 in Kaggle**

         * Best k in range 10-30 was k=20

         * Best Validation Accuracy for k=20, 0.56

         * F1 Score (for validation data) for k=20: 0.49

         * Training data Accuracy for k = 20: 0.59382

     * Increasing the sample size of training data for training to be 400 000 (sample of 400 000 to all of the data)

         * **We got f1 score of 0.26029 in Kaggle**

         * Best k in range 10-40 was k=35

         * Best Validation Accuracy for k=35, approx. 0.595

         * F1 Score (for validation data) for k=35: 0.5375

         * Training data Accuracy for k = 35: 0.6175



 * **Scale categorial features as well**

     * Let's use standardization to also standardize the categorial values instead of only metric columns.

         * We will experience this with size of training data as 400 000 (sample of 400 000 to all of the data)

         * Let's try 10 different k values around the k=35 (so from k=30 to k=40)

         * Results:

             * **We got f1 score of 0.27128 in Kaggle**

             * Best k in range 30-40: 37

             * Best Validation Accuracy k=37: 0.648

             * F1 Score for k=37: 0.591

             * Training Accuracy for k=37 in range 30-40: 0.659315



 ### Future optimizations for project part 2

 * **Let go more features**

     * Get rid of highly correlating features with each other (WCIO... features)

     * Add Average Weekly Wage as it might be important

     * Try optimal features from LogisticRegression's RFE.

     * Implement KNNClassifier with RFE

 * Oversample minority target variable values

 * Different PCA values or without PCA

 * Weighting closer neighbors more

 * KFold

 ## Testing of almost random Classifier

 * Just for fun of it, random classifier was used. Guessing was weighted to favor most usual value in target variable in training data. Only other value we allowed the model to guess was the second most usual value in target variable in training data.



 ## Result: Bad as expected

 * In kaggle, f1 score of 0.13 was achieved

In [ ]:
# %%
# NOTE: Commented out since this is not needed for achieving our final results

# Find two most usual target values
# print(train_data['Claim Injury Type'].value_counts())

# Check label mapping
# print(claim_injury_type_mapping)

# Take the Claim Identifier numbers from model1.csv
# model1 = pd.read_csv('model1.csv')
# print(model1['Claim Identifier'])

# # Make a new submission file (model2.csv) with the two most usual target values randomly assigned for every Claim Identifier in model1.csv wtih more ephasis for the most usual value


# # Create a list of the two most usual target values
# most_usual_values = [0, 1]

# # Generate random guesses with weighted probabilities

# # Define the probabilities for each value
# probabilities = [0.8, 0.2]


# print(label_mapping)

# # Generate random guesses
# random_guesses = np.random.choice(most_usual_values, size=len(model1), p=probabilities)

# # Create a new DataFrame for the submission
# submission_model2 = pd.DataFrame({
#     'Claim Identifier': model1['Claim Identifier'],
#     'Claim Injury Type': random_guesses
# })

# # Map the values in submission_model2 'Claim Injury Type' to match the original target values in label_mapping
# submission_model2['Claim Injury Type'] = submission_model2['Claim Injury Type'].map(label_mapping)
# print(submission_model2)


# # Save the new submission file
# submission_model2.to_csv('model3.csv', index=False)




 # Things to  consider when optimizing models in deliverable 2



 ## Add Average Weekly Wage back with proper null value imputations



 ## These columns show low correlation with the target variable. Consider training models with and without them to see if they affect accuracy, precision and f1-score

 * District Name

 * Gender

 * Carrier type

 * Number of Dependents



 ## These columns showed high correlation between each other. Consider dropping one or two of them

 * WCIO Cause of Injury Code, WCIO Nature of Injury Code and WCIO Part Of Body Code had high correlations between each other AND nice correlations between target variable.

     * We can consider droppig one or two of these to avoid redundancy



 ## Scaling

 * Try to also change z-score scaling to min-max scaling and see if it affects the accuracy of the model

 * Experience with scaling only numerical values and both numerical and categorial values



 ## Outliers

 * See if thresholding the days until injury reported -feature's values to even lower max-value affects the model accuracy





 ## Keep in mind, the unbalance of the target variable categories. If model performance is not good, try oversampling the rows with minority Claim Injury Type values or try SMOTE

 Claim Injury Type

 - 2. NON-COMP        0.510175

 - 4. TEMPORARY       0.257070

 - 3. MED ONLY        0.121362

 - 5. PPD SCH LOSS    0.084481

 - 1. CANCELLED       0.018900

 - 6. PPD NSL         0.007112

 - 8. DEATH           0.000775

 - 7. PTD             0.000126

 # OPTIMIZATIONS ON KNNCLASSIFIER

In [ ]:
# %%
#NOTE / TODO: START FROM PCA ANALYSIS WHAT WAS DONE IN DM LAB 08!! CHOOSE HOW MANY PCA's TO KEEP

# Some heuristics (rules of thumb):

# Reference: **Jolliffe, I. T. (2002). Principal component analysis.**

# Section 6.1: *How Many Principal Components?*

# - 6.1.1. Cumulative percentage of total variance
#     - Keep 70-80% of total variance
# - 6.1.2. Kaiser's rule
#     - Keep PC with variance >= 1
# - 6.1.3. Scree plot
    # - Use elbow method


# FROM DM LAB 08

# # Get PCA output as table

# # Get the eigenvalues (explained variance)
# explained_variance = pca.explained_variance_

# # Get the explained variance ratio
# explained_variance_ratio = pca.explained_variance_ratio_

# # get the cumulative explained variance ratio 
# # Hint: use np.cumsum()
# cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)

# # Combine into a dataframe
# pca_results = pd.DataFrame(
#     {
#         "Eigenvalue": explained_variance,
#         "Difference": np.insert(np.diff(explained_variance), 0, 0),
#         "Proportion": explained_variance_ratio,
#         "Cumulative": cumulative_explained_variance_ratio
#     },
#         index=range(1, pca.n_components_ + 1)
# )

# pca_results




 # DEEP NEURAL NETWORK MODEL TRAINING AND OPTIMIZATION

In [ ]:
# %%
train_data_NN = train_data_standardized.copy()
test_data_NN = test_data_standardized.copy()

# Prepare the data
predictor_cols = train_data_NN.columns[train_data_NN.columns != "Claim Injury Type"]
X = train_data_NN[predictor_cols]
y = train_data_NN["Claim Injury Type"]


# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5, stratify=y)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the total number of features in the dataset
num_features = X.shape[1]

# Lists to store the results for plotting
accuracy_scores_train = []
accuracy_scores_val = []
f1_scores_val = []
num_feature_list = list(range(1, num_features + 1))

# Use LogisticRegression as the base estimator for RFE
base_estimator = LogisticRegression(max_iter=1000)  

# Loop over feature numbers from 1 to the total number of features
for n_features in num_feature_list:
    # Use RFE to select n_features
    rfe = RFE(estimator=base_estimator, n_features_to_select=n_features)
    rfe.fit(X_train, y_train)
    selected_features = rfe.support_

    # Select the features from training and validation sets
    X_train_rfe = X_train[:, selected_features]
    X_val_rfe = X_val[:, selected_features]
    
    # Build the MLPClassifier with a deep structure and small hidden layers
    mlp = MLPClassifier(
        hidden_layer_sizes=(32, 16, 16, 8, 8, 4),  # Deep structure with gradually decreasing sizes
        activation='relu',
        solver='adam',
        max_iter=110  # Increase this if convergence warnings appear (UPDATE: THIS WAS 90 AND I GOT CONVERGING ERRORS)
    )

    # Train the MLP model
    mlp.fit(X_train_rfe, y_train)

    # Evaluate on training data
    train_preds = mlp.predict(X_train_rfe)
    accuracy_train = accuracy_score(y_train, train_preds)

    # Evaluate on validation data
    val_preds = mlp.predict(X_val_rfe)
    accuracy_val = accuracy_score(y_val, val_preds)
    f1_val = f1_score(y_val, val_preds, average='weighted')

    # Store results
    accuracy_scores_train.append(accuracy_train)
    accuracy_scores_val.append(accuracy_val)
    f1_scores_val.append(f1_val)

    # Get selected feature names
    selected_features_names = X.columns[selected_features]
    # Print results for each feature set
    print(f"Number of features: {n_features}")
    print("Selected features: ", selected_features_names)
    print(f"Training Accuracy: {accuracy_train}")
    print(f"Validation Accuracy: {accuracy_val}")
    print(f"Validation F1 Score: {f1_val}")
    print("-" * 50)

# Plotting accuracy and F1 scores for different feature counts
plt.figure(figsize=(12, 6))

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(num_feature_list, accuracy_scores_train, label='Training Accuracy', marker='o')
plt.plot(num_feature_list, accuracy_scores_val, label='Validation Accuracy', marker='x')
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy vs. Number of Features')
plt.legend()

# Plot Validation F1 Score
plt.subplot(1, 2, 2)
plt.plot(num_feature_list, f1_scores_val, label='Validation F1 Score', marker='^')
plt.xlabel('Number of Features')
plt.ylabel('F1 Score')
plt.title('Validation F1 Score vs. Number of Features')
plt.legend()

plt.tight_layout()
plt.show()




 ## NN Model



 ### Train two models with two most promising set of features and and make predictions to test data with them

 * Set of features chosen are:

     * 8 features (this far, almost all (expect one) features have been )

         * Selected features:  Index(['Accident Date', 'Assembly Date', 'Attorney/Representative',

        'Average Weekly Wage', 'C-2 Date', 'IME-4 Count',

        'WCIO Nature of Injury Code', 'WCIO Part Of Body Code'],

       dtype='object')

         * Training Accuracy: 0.7463289006068101

         * Validation Accuracy: 0.7439138195029581

         * Validation F1 Score: 0.689082918712831

     * 11 features (no further improvement worth mentioning in accuracy nor f1-score)

         * Selected features:  Index(['Accident Date', 'Age at Injury', 'Assembly Date',

        'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date', 'Gender',

        'IME-4 Count', 'Industry Code', 'WCIO Nature of Injury Code',

        'WCIO Part Of Body Code'],

         dtype='object')

         * Training Accuracy: 0.76391031260122

         * Validation Accuracy: 0.761227836517989

         * Validation F1 Score: 0.7046574402209247

 * NOTE: We didn't try 4 features yet, even though it looked promising, since two of them would have been date columns. Later features are assessed again with focus on other than date columns

 * These feature sets were chosen based on highest validation accuracy and highest f1-score

 * For all feature sets, validation and testing data accuracy are very close to each other.

In [ ]:
# %%
# Build model with the optimal number of features

train_data_NN = train_data_non_standardized.copy()
test_data_NN = test_data_non_standardized.copy()

# Prepare the data (NOTE: Object names are wrong and misleading here, since after adding more features, the optimal number of features changed)

# THIS predictor_cols_13 IS ACTUALLY SET OF 11 FEATURES
predictor_cols_13 = ['Accident Date', 'Age at Injury', 'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage', 'C-2 Date', 'Gender', 'IME-4 Count', 'Industry Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

# THIS predictor_cols_10 IS ACTUALLY SET OF 8 FEATURES
predictor_cols_10 = ['Accident Date', 'Assembly Date', 'Attorney/Representative','Average Weekly Wage', 'C-2 Date', 'IME-4 Count','WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

X_13 = train_data_NN[predictor_cols_13]
X_10 = train_data_NN[predictor_cols_10]

y_13 = train_data_NN["Claim Injury Type"]
y_10 = train_data_NN["Claim Injury Type"]

# Split data into training and validation sets
X_train_13, X_val_13, y_train_13, y_val_13 = train_test_split(X_13, y_13, test_size=0.2, random_state=5, stratify=y_13)
X_train_10, X_val_10, y_train_10, y_val_10 = train_test_split(X_10, y_10, test_size=0.2, random_state=5, stratify=y_10)

# Standardize the data
scaler_13 = StandardScaler()
scaler_10 = StandardScaler()

X_train_13 = scaler_13.fit_transform(X_train_13)
X_val_13 = scaler_13.transform(X_val_13)
test_data_NN_13 = test_data_NN[predictor_cols_13]
# Scale the test data
test_data_NN_13 = scaler_13.transform(test_data_NN_13)

X_train_10 = scaler_10.fit_transform(X_train_10)
X_val_10 = scaler_10.transform(X_val_10)
test_data_NN_10 = test_data_NN[predictor_cols_10]
# Scale the test data
test_data_NN_10 = scaler_10.transform(test_data_NN_10)

# Build the MLPClassifier with a deep structure and small hidden layers
nn_model_10_features = MLPClassifier(
    hidden_layer_sizes=(32, 16, 16, 8, 8, 4),
    activation='relu',
    solver='adam',
    max_iter=150 
)

nn_model_13_features = MLPClassifier(
    hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
    activation='relu',
    solver='adam',
    max_iter=150
)


# Train the MLP models
nn_model_10_features.fit(X_train_10, y_train_10)
nn_model_13_features.fit(X_train_13, y_train_13)

# Evaluate on training data
train_preds_10 = nn_model_10_features.predict(X_train_10)
accuracy_train_10 = accuracy_score(y_train_10, train_preds_10)
print(f"Training Accuracy with 8 features: {accuracy_train_10}")

# Evaluate on validation data
val_preds_10 = nn_model_10_features.predict(X_val_10)
accuracy_val_10 = accuracy_score(y_val_10, val_preds_10)
f1_val_10 = f1_score(y_val_10, val_preds_10, average='weighted')
print(f"Validation Accuracy with 8 features: {accuracy_val_10}")
print(f"Validation F1 Score with 8 features: {f1_val_10}")

# Evaluate precision on validation data
precision_val_10 = precision_score(y_val_10, val_preds_10, average='weighted')
print(f"Validation Precision Score with 8 features: {precision_val_10}")

# Same for 13 features
train_preds_13 = nn_model_13_features.predict(X_train_13)
accuracy_train_13 = accuracy_score(y_train_13, train_preds_13)
print(f"Training Accuracy with 11 features: {accuracy_train_13}")

val_preds_13 = nn_model_13_features.predict(X_val_13)
accuracy_val_13 = accuracy_score(y_val_13, val_preds_13)
f1_val_13 = f1_score(y_val_13, val_preds_13, average='weighted')
print(f"Validation Accuracy with 11 features: {accuracy_val_13}")
print(f"Validation F1 Score with 11 features: {f1_val_13}")

precision_val_13 = precision_score(y_val_13, val_preds_13, average='weighted')
print(f"Validation Precision Score with 11 features: {precision_val_13}")



In [ ]:
# %%
# Save models

# Save the models
joblib.dump(nn_model_10_features, 'nn_model_8_features.pkl')
joblib.dump(nn_model_13_features, 'nn_model_11_features.pkl')


In [ ]:
# %%
# Get predictions for the test data with both models
test_preds_10 = nn_model_10_features.predict(test_data_NN_10)
test_preds_13 = nn_model_13_features.predict(test_data_NN_13)


In [ ]:
# %%
# See the unique values in the predictions
unique_values = np.unique(test_preds_10)
print(unique_values)

unique_values_13 = np.unique(test_preds_13)
print(unique_values_13)

# Format the predictions according to the required label format
formatted_predictions_10 = [claim_injury_type_mapping[pred] for pred in test_preds_10]
formatted_predictions_13 = [claim_injury_type_mapping[pred] for pred in test_preds_13]


# Create the submission DataFrames
submission_10 = pd.DataFrame({
    "Claim Identifier": test_data.index,
    "Claim Injury Type": formatted_predictions_10
})

submission_13 = pd.DataFrame({
    "Claim Identifier": test_data.index,
    "Claim Injury Type": formatted_predictions_13
})


# Save to CSV in the required format
submission_10.to_csv("Group39__Version10.csv", index=False)
submission_13.to_csv("Group39__Version11.csv", index=False)

# Check length of the submission files
print(len(submission_10))
print(len(submission_13))


 ### Result in kaggle.

 * 8 features: 0.30994

 * 10 features: 0.19609

 ## Further NN Model Experiments: Feature Selection re-visited:

 * Train model inside RFE:

     * Without Accident Date and Assembly date, since their difference is already presented in Days Untill Injury reported column

         * We will try this, since we don't think that the day of the injury itself or Assembly date alone are very effective predictors for the inury type of the claim

     * We will use more iterations for NN model, since er didn't fully converge after 5 features

In [ ]:
# %%
train_data_NN = train_data_non_standardized.copy()
test_data_NN = test_data_non_standardized.copy()

# Prepare the data
predictor_cols = train_data_NN.columns.difference(["Claim Injury Type", "Accident Date", "Assembly Date"])
print(predictor_cols)

X = train_data_NN[predictor_cols]
y = train_data_NN["Claim Injury Type"]


# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5, stratify=y)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the total number of features in the dataset
num_features = X.shape[1]

# Lists to store the results for plotting
accuracy_scores_train = []
accuracy_scores_val = []
f1_scores_val = []
precision_val = []
num_feature_list = list(range(1, num_features + 1))

# Use LogisticRegression as the base estimator for RFE
base_estimator = LogisticRegression(max_iter=1000)  

# Loop over feature numbers from 1 to the total number of features
for n_features in num_feature_list:
    # Use RFE to select n_features
    rfe = RFE(estimator=base_estimator, n_features_to_select=n_features)
    rfe.fit(X_train, y_train)
    selected_features = rfe.support_

    # Select the features from training and validation sets
    X_train_rfe = X_train[:, selected_features]
    X_val_rfe = X_val[:, selected_features]
    
    # Build the MLPClassifier with a deep structure and small hidden layers
    mlp = MLPClassifier(
        hidden_layer_sizes=(32, 16, 16, 8, 8, 4),  # Deep structure with gradually decreasing sizes
        activation='relu',
        solver='adam',
        max_iter=200
    )

    # Train the MLP model
    mlp.fit(X_train_rfe, y_train)

    # Evaluate on training data
    train_preds = mlp.predict(X_train_rfe)
    accuracy_train = accuracy_score(y_train, train_preds)

    # Evaluate on validation data
    val_preds = mlp.predict(X_val_rfe)
    accuracy_val = accuracy_score(y_val, val_preds)
    f1_val = f1_score(y_val, val_preds, average='weighted')

    # Evaluate precision on validation data
    precision_on_val = precision_score(y_val, val_preds, average='weighted')

    # Store results
    accuracy_scores_train.append(accuracy_train)
    accuracy_scores_val.append(accuracy_val)
    f1_scores_val.append(f1_val)
    precision_val.append(precision_on_val)

    # Get selected feature names
    selected_features_names = X.columns[selected_features]
    # Print results for each feature set
    print(f"Number of features: {n_features}")
    print("Selected features: ", selected_features_names)
    print(f"Training Accuracy: {accuracy_train}")
    print(f"Validation Accuracy: {accuracy_val}")
    print(f"Validation F1 Score: {f1_val}")
    print(f"Validation Precision Score: {precision_on_val}")
    print("-" * 50)

# Plotting accuracy and F1 scores for different feature counts
plt.figure(figsize=(12, 6))

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(num_feature_list, accuracy_scores_train, label='Training Accuracy', marker='o')
plt.plot(num_feature_list, accuracy_scores_val, label='Validation Accuracy', marker='x')
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy vs. Number of Features')
plt.legend()

# Plot Validation F1 Score and Precision
plt.subplot(1, 2, 2)
plt.plot(num_feature_list, f1_scores_val, label='Validation F1 Score', marker='^')
plt.plot(num_feature_list, precision_val, label='Validation Precision', marker='s')
plt.xlabel('Number of Features')
plt.ylabel('Score')
plt.title('Validation F1 Score and Precision vs. Number of Features')
plt.legend()
plt.tight_layout()
plt.show()




 # JÄIT TÄHÄN WAGEN KANSSA!!



 ## - JATKA

 ## - JATKA

 ## - JATKA

 ## - JATKA

 ## - JATKA

 ## Try two most promising set of features and make Kaggle returns with them

 ### Optimal number of features is now .... Let's build the model with a bit more max iterations for this set of ... features, make predictions for test data and let's try kaggle return.

In [ ]:
# %%
# Build model with the optimal number of features

train_data_NN = train_data_non_standardized.copy()
test_data_NN = test_data_non_standardized.copy()

# Prepare the data
predictor_cols_11 = ['Age at Injury', 'Attorney/Representative', 'C-2 Date', 'Carrier Type', 'Days Until Injury Reported', 'Gender', 'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code','WCIO Part Of Body Code']

X_11 = train_data_NN[predictor_cols_11]

y_11 = train_data_NN["Claim Injury Type"]

# Split data into training and validation sets
X_train_11, X_val_11, y_train_11, y_val_11 = train_test_split(X_11, y_11, test_size=0.2, random_state=5, stratify=y_11)

# Standardize the data
scaler_11 = StandardScaler()

X_train_11 = scaler_11.fit_transform(X_train_11)
X_val_11 = scaler_11.transform(X_val_11)
test_data_NN_11 = test_data_NN[predictor_cols_11]
# Scale the test data
test_data_NN_11 = scaler_11.transform(test_data_NN_11)


nn_model_11_features = MLPClassifier(
    hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
    activation='relu',
    solver='adam',
    max_iter=200  
)


# Train the MLP model
nn_model_11_features.fit(X_train_11, y_train_11)

# Evaluate on training data
train_preds_11 = nn_model_11_features.predict(X_train_11)
accuracy_train_11 = accuracy_score(y_train_11, train_preds_11)
print(f"Training Accuracy with 11 features (Assembly Date and Accident Date excluded): {accuracy_train_11}")

# Evaluate on validation data
val_preds_11 = nn_model_11_features.predict(X_val_11)
accuracy_val_11 = accuracy_score(y_val_11, val_preds_11)
f1_val_11 = f1_score(y_val_11, val_preds_11, average='weighted')
print(f"Validation Accuracy with 11 features (Assembly Date and Accident Date excluded): {accuracy_val_11}")
print(f"Validation F1 Score with 11 features (Assembly Date and Accident Date excluded): {f1_val_11}")

# Evaluate precision on validation data
precision_val_11 = precision_score(y_val_11, val_preds_11, average='weighted')
print(f"Validation Precision Score with 11 features (Assembly Date and Accident Date excluded): {precision_val_11}")




In [ ]:
# %%
# Save models

# Save the models
joblib.dump(nn_model_11_features, 'nn_model_11_features.pkl')


In [ ]:
# %%
# Get predictions for the test data with both models
test_preds_11 = nn_model_11_features.predict(test_data_NN_11)

# See the unique values in the predictions
unique_values = np.unique(test_preds_11)
print(unique_values)

# Format the predictions according to the required label format
formatted_predictions_11 = [claim_injury_type_mapping[pred] for pred in test_preds_11]

# Create the submission DataFrames
submission_11 = pd.DataFrame({
    "Claim Identifier": test_data.index,
    "Claim Injury Type": formatted_predictions_11
})


# Save to CSV in the required format
submission_11.to_csv("Group39__Version11.csv", index=False)

# Check length of the submission files
print(len(submission_11))


 #### Result: 0.275 f1 score in Kaggle

 ## OUTDATED!: (CODE ABOUT SMOTE COMMENTED OUT AND TRIED AFTER IMPORTANT FIXING OF PREPROCESSING )Further NN Model Experiments: Target value imbalance

 * Train model inside RFE and then with 3 best sets of features found this far (according to local environment assessment as well as Kaggle competition f1 score)

     * Before training, apply Oversampling, Undersampling or SMOTE Techniques to get more minority result class values to the training. Our model is probably not good in predicting for those minoriity target variable value classes.

 * Training iterations of predictor features:

     * Inside RFE to find best set of features:

         * Inside RFE without assembly and accident date

     * Predict for two best sets from both RFE procesure above

     * Best 3 sets of previous experiments:

         * See if modifications to SMOTE are needed





 ## Chosen method: Creating synthetic minority class data with SMOTE

 * We chose SMOTE and not oversampling, since duplicating minority class members a lot, we might add risk for overfitting

In [ ]:
# %%
# # Check the target value counts in train data
# train_data_balanced = train_data_non_standardized.copy()


# train_data_balanced['Claim Injury Type'].value_counts()


# # Prepare the data
# predictor_cols = train_data_balanced.columns[train_data_balanced.columns != "Claim Injury Type"]
# X = train_data_balanced[predictor_cols]
# y = train_data_balanced["Claim Injury Type"]

# # Scale the data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)


# # We set a custom sampling strategy to reduce the amount of created syntetic samples and to keep the size of dataset OK 
# # From value count we saw that Majority class has 282253 samples. For minority classes that have less than 25% of the majority class samples are 3,5,1,6,8 and 7. For those classes we will set the sampling strategy to 0.25
# # So we are adding synthetic samples to minority classes to have 25% of the majority class samples

# """
# value counts
# 2    282253
# 4    141022
# 3     67250
# 5     46867
# 1     10058
# 6      3945
# 8       431
# 7        69
# """

# target_size = int(0.25 * 282253)

# # Define sampling strategy as a dictionary
# # Only include classes with fewer samples than target_size (70563). 
# sampling_strategy = {
#     3: target_size,  # Currently 67250 -> Increase to 70563
#     5: target_size,  # Currently 46867 -> Increase to 70563
#     1: target_size,  # Currently 10058 -> Increase to 70563
#     6: target_size,  # Currently 3945 -> Increase to 70563
#     7: target_size,  # Currently 69 -> Increase to 70563
#     8: target_size,  # Currently 431 -> Increase to 70563
# }



# # Apply SMOTE 
# smote = SMOTE(sampling_strategy=sampling_strategy, random_state=5)
# X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# # Convert the resampled data back to a DataFrame
# X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
# y_resampled_df = pd.DataFrame(y_resampled, columns=['Claim Injury Type'])
# train_data_resampled = pd.concat([X_resampled_df, y_resampled_df], axis=1)

# # Check for lenght of data and value counts in the resampled data
# print(len(train_data_resampled))
# train_data_resampled['Claim Injury Type'].value_counts()




 ## Model Training with new balanced training data

In [ ]:
# %%
# # RFE WITH ALL FEATURES (for SMOTE'd data)

# train_data_NN = train_data_resampled.copy()
# test_data_NN  = test_data_non_standardized.copy()


# # Prepare the data
# predictor_cols = train_data_NN.columns.difference(["Claim Injury Type", "Accident Date", "Assembly Date"])
# X = train_data_NN[predictor_cols]
# y = train_data_NN["Claim Injury Type"]


# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5, stratify=y)

# # Define the total number of features in the dataset
# num_features = X.shape[1]





In [ ]:
# %%

# # Lists to store the results for plotting
# accuracy_scores_train = []
# accuracy_scores_val = []
# f1_scores_val = []
# num_feature_list = list(range(1, num_features + 1))

# # Use LogisticRegression as the base estimator for RFE
# base_estimator = LogisticRegression(max_iter=1000)  

# # Loop over feature numbers from 1 to the total number of features
# for n_features in num_feature_list:
#     # Use RFE to select n_features
#     rfe = RFE(estimator=base_estimator, n_features_to_select=n_features)
#     rfe.fit(X_train, y_train)
#     selected_features = rfe.support_

#     #NOTE: THIS FOR NUMPY!
#     # Select the features from training and validation sets
#     # X_train_rfe = X_train[:, selected_features]
#     # X_val_rfe = X_val[:, selected_features]

#     #NOTE: THIS FOR DATAFRAMES!
#     selected_columns = X_train.columns[selected_features] # Get the selected column names
#     X_train_rfe = X_train[selected_columns]
#     X_val_rfe = X_val[selected_columns]

    
#     # Build the MLPClassifier with a deep structure and small hidden layers
#     mlp = MLPClassifier(
#         hidden_layer_sizes=(32, 16, 16, 8, 8, 4),  # Deep structure with gradually decreasing sizes
#         activation='relu',
#         solver='adam',
#         max_iter=200  
#     )

#     # Train the MLP model
#     mlp.fit(X_train_rfe, y_train)

#     # Evaluate on training data
#     train_preds = mlp.predict(X_train_rfe)
#     accuracy_train = accuracy_score(y_train, train_preds)

#     # Evaluate on validation data
#     val_preds = mlp.predict(X_val_rfe)
#     accuracy_val = accuracy_score(y_val, val_preds)
#     f1_val = f1_score(y_val, val_preds, average='weighted')

#     # Store results
#     accuracy_scores_train.append(accuracy_train)
#     accuracy_scores_val.append(accuracy_val)
#     f1_scores_val.append(f1_val)

#     # Get selected feature names
#     selected_features_names = selected_columns
#     # Print results for each feature set
#     print(f"Number of features: {n_features}")
#     print("Selected features: ", selected_features_names)
#     print(f"Training Accuracy: {accuracy_train}")
#     print(f"Validation Accuracy: {accuracy_val}")
#     print(f"Validation F1 Score: {f1_val}")
#     print("-" * 50)

# # Plotting accuracy and F1 scores for different feature counts
# plt.figure(figsize=(12, 6))

# # Plot Training and Validation Accuracy
# plt.subplot(1, 2, 1)
# plt.plot(num_feature_list, accuracy_scores_train, label='Training Accuracy', marker='o')
# plt.plot(num_feature_list, accuracy_scores_val, label='Validation Accuracy', marker='x')
# plt.xlabel('Number of Features')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy vs. Number of Features')
# plt.legend()

# # Plot Validation F1 Score
# plt.subplot(1, 2, 2)
# plt.plot(num_feature_list, f1_scores_val, label='Validation F1 Score', marker='^')
# plt.xlabel('Number of Features')
# plt.ylabel('F1 Score')
# plt.title('Validation F1 Score vs. Number of Features')
# plt.legend()

# plt.tight_layout()
# plt.show()




 Next we will train models with 4 most promising set of features with even more iterations allowed for the NN and save the model.

 * Set 1: Three features: ['Attorney/Representative', 'IME-4 Count', 'WCIO Nature of Injury Code']

     * We want to explore this since it was the first set of features that pumped up the accuracy radically

 * Set 2: Nine features: ['Age at Injury', 'Attorney/Representative', 'C-2 Date', 'Days Until Injury Reported', 'Gender', 'IME-4 Count', 'Industry Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']

     * We want to explore this since it was the set of features, which after, we didn't significally prove accuracy or f1-score

 * Set 3: Thirteen features: All but Assemly Date and Age at Injury

     * We want to explore this since it was the set that produced overall highest accuracies and f1-scores

 * Set 4: All the features that improved training and validation accracy and f1-score:

     * Selected features: ['Age at Injury', 'Attorney/Representative', 'C-2 Date', 'Days Until Injury Reported', 'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Carrier Type']

     * We want to explore this since we want to try how the result comes up with only the features that improved the metrics from the previous iteration of RFE process

In [ ]:
# %%
# # Apply smote to all datasets

# # Check the target value counts in train data
# train_data_balanced = train_data_non_standardized.copy()
# test_data_balanced = test_data_non_standardized.copy()

# # Prepare the data
# predictor_cols_all = train_data_NN.columns.difference(["Claim Injury Type", "Accident Date", "Assembly Date"])
# predictor_cols_3 = ['Attorney/Representative', 'IME-4 Count', 'WCIO Nature of Injury Code'] 
# predictor_cols_9 = ['Age at Injury', 'Attorney/Representative', 'C-2 Date', 'Days Until Injury Reported', 'Gender', 'IME-4 Count', 'Industry Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code']
# predictor_cols_improved = ['Age at Injury', 'Attorney/Representative', 'C-2 Date', 'Days Until Injury Reported', 'IME-4 Count', 'Industry Code', 'WCIO Cause of Injury Code', 'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Carrier Type']


# X_all = train_data_NN[predictor_cols_all]
# X_3 = train_data_NN[predictor_cols_3]
# X_9 = train_data_NN[predictor_cols_9]
# X_improved = train_data_NN[predictor_cols_improved]

# y_all = train_data_NN["Claim Injury Type"]
# y_3 = train_data_NN["Claim Injury Type"]
# y_9 = train_data_NN["Claim Injury Type"]
# y_improved = train_data_NN["Claim Injury Type"]



# # Scale the data
# scaler_all = StandardScaler()
# scaler_3 = StandardScaler()
# scaler_9 = StandardScaler()
# scaler_improved = StandardScaler()

# X_scaled_all = scaler_all.fit_transform(X_all)
# X_scaled_3 = scaler_3.fit_transform(X_3)
# X_scaled_9 = scaler_9.fit_transform(X_9)
# X_scaled_improved = scaler_improved.fit_transform(X_improved)

# # Prepeare test data
# test_data_NN_all = test_data_NN[predictor_cols_all]
# test_data_NN_3 = test_data_NN[predictor_cols_3]
# test_data_NN_9 = test_data_NN[predictor_cols_9]
# test_data_NN_improved = test_data_NN[predictor_cols_improved]

# # Scale the test data
# test_data_NN_all = scaler_all.transform(test_data_NN_all)
# test_data_NN_3 = scaler_3.transform(test_data_NN_3)
# test_data_NN_9 = scaler_9.transform(test_data_NN_9)
# test_data_NN_improved = scaler_improved.transform(test_data_NN_improved)

# # We set a custom sampling strategy to reduce the amount of created syntetic samples and to keep the size of dataset OK 
# # From value count we saw that Majority class has 282253 samples. For minority classes that have less than 25% of the majority class samples are 3,5,1,6,8 and 7. For those classes we will set the sampling strategy to 0.25
# # So we are adding synthetic samples to minority classes to have 25% of the majority class samples

# """
# value counts
# 2    282253
# 4    141022
# 3     67250
# 5     46867
# 1     10058
# 6      3945
# 8       431
# 7        69
# """

# target_size = int(0.25 * 282253)

# # Define sampling strategy as a dictionary
# # Only include classes with fewer samples than target_size (70563). 
# sampling_strategy = {
#     3: target_size,  # Currently 67250 -> Increase to 70563
#     5: target_size,  # Currently 46867 -> Increase to 70563
#     1: target_size,  # Currently 10058 -> Increase to 70563
#     6: target_size,  # Currently 3945 -> Increase to 70563
#     7: target_size,  # Currently 69 -> Increase to 70563
#     8: target_size,  # Currently 431 -> Increase to 70563
# }



# # Apply SMOTE 
# smote_all = SMOTE(sampling_strategy=sampling_strategy, random_state=5)
# smote_3 = SMOTE(sampling_strategy=sampling_strategy, random_state=5)
# smote_9 = SMOTE(sampling_strategy=sampling_strategy, random_state=5)
# smote_improved = SMOTE(sampling_strategy=sampling_strategy, random_state=5)

# X_resampled_all, y_resampled_all = smote_all.fit_resample(X_scaled_all, y_all)
# X_resampled_3, y_resampled_3 = smote_3.fit_resample(X_scaled_3, y_3)
# X_resampled_9, y_resampled_9 = smote_9.fit_resample(X_scaled_9, y_9)
# X_resampled_improved, y_resampled_improved = smote_improved.fit_resample(X_scaled_improved, y_improved)

# # Convert the resampled data back to a DataFrame

# X_resampled_all_df = pd.DataFrame(X_resampled_all, columns=X_all.columns)
# y_resampled_all_df = pd.DataFrame(y_resampled_all, columns=['Claim Injury Type'])
# train_data_resampled_all = pd.concat([X_resampled_all_df, y_resampled_all_df], axis=1)

# X_resampled_3_df = pd.DataFrame(X_resampled_3, columns=X_3.columns)
# y_resampled_3_df = pd.DataFrame(y_resampled_3, columns=['Claim Injury Type'])
# train_data_resampled_3 = pd.concat([X_resampled_3_df, y_resampled_3_df], axis=1)

# X_resampled_9_df = pd.DataFrame(X_resampled_9, columns=X_9.columns)
# y_resampled_9_df = pd.DataFrame(y_resampled_9, columns=['Claim Injury Type'])
# train_data_resampled_9 = pd.concat([X_resampled_9_df, y_resampled_9_df], axis=1)

# X_resampled_improved_df = pd.DataFrame(X_resampled_improved, columns=X_improved.columns)
# y_resampled_improved_df = pd.DataFrame(y_resampled_improved, columns=['Claim Injury Type'])
# train_data_resampled_improved = pd.concat([X_resampled_improved_df, y_resampled_improved_df], axis=1)



# # Check for lenght of data and value counts in the resampled data
# print(len(train_data_resampled_all))
# print(len(train_data_resampled_3))
# print(len(train_data_resampled_9))
# print(len(train_data_resampled_improved))

# print(train_data_resampled_all['Claim Injury Type'].value_counts())
# print(train_data_resampled_3['Claim Injury Type'].value_counts())
# print(train_data_resampled_9['Claim Injury Type'].value_counts())
# print(train_data_resampled_improved['Claim Injury Type'].value_counts())




In [ ]:
# %%
# # Split data into training and validation sets
# X_resampled_all_df = train_data_resampled_all[predictor_cols_all]
# y_resampled_all_df = train_data_resampled_all["Claim Injury Type"]
# X_train_all, X_val_all, y_train_all, y_val_all = train_test_split(X_resampled_all_df, y_resampled_all_df, test_size=0.2, random_state=5, stratify=y_resampled_all_df)

# X_resampled_3_df = train_data_resampled_3[predictor_cols_3]
# y_resampled_3_df = train_data_resampled_3["Claim Injury Type"]
# X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X_resampled_3_df, y_resampled_3_df, test_size=0.2, random_state=5, stratify=y_resampled_3_df)

# X_resampled_9_df = train_data_resampled_9[predictor_cols_9]
# y_resampled_9_df = train_data_resampled_9["Claim Injury Type"]
# X_train_9, X_val_9, y_train_9, y_val_9 = train_test_split(X_resampled_9_df, y_resampled_9_df, test_size=0.2, random_state=5, stratify=y_resampled_9_df)

# X_resampled_improved_df = train_data_resampled_improved[predictor_cols_improved]
# y_resampled_improved_df = train_data_resampled_improved["Claim Injury Type"]
# X_train_improved, X_val_improved, y_train_improved, y_val_improved = train_test_split(X_resampled_improved_df, y_resampled_improved_df, test_size=0.2, random_state=5, stratify=y_resampled_improved_df)


# # Define the total number of features in the dataset
# num_features_all = X_resampled_all_df.shape[1]
# num_features_3 = X_resampled_3_df.shape[1]
# num_features_9 = X_resampled_9_df.shape[1]
# num_features_improved = X_resampled_improved_df.shape[1]



In [ ]:
# %%
# # Train model with all features with more iterations
# nn_model_all_features = MLPClassifier(
#     hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
#     activation='relu',
#     solver='adam',
#     max_iter=300  
# )

# # Train the MLP model
# nn_model_all_features.fit(X_train_all, y_train_all)

# # Evaluate on training data
# train_preds_all = nn_model_all_features.predict(X_train_all)
# accuracy_train_all = accuracy_score(y_train_all, train_preds_all)
# print(f"Training Accuracy with all features (Other than Assembly and Accident Date): {accuracy_train_all}")

# # Evaluate on validation data
# val_preds_all = nn_model_all_features.predict(X_val_all)
# accuracy_val_all = accuracy_score(y_val_all, val_preds_all)
# f1_val_all = f1_score(y_val_all, val_preds_all, average='weighted')
# print(f"Validation Accuracy with all features (Other than Assembly and Accident Date): {accuracy_val_all}")
# print(f"Validation F1 Score with all features (Other than Assembly and Accident Date): {f1_val_all}")

# # Evaluate precision on validation data
# precision_val_all = precision_score(y_val_all, val_preds_all, average='weighted')
# print(f"Validation Precision Score with all features (Other than Assembly and Accident Date): {precision_val_all}")

# # Save the model
# joblib.dump(nn_model_all_features, 'nn_model_all_features_smote.pkl')


In [ ]:
# %%
# # Train model with 3 features with more iterations
# nn_model_3_features = MLPClassifier(
#     hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
#     activation='relu',
#     solver='adam',
#     max_iter=300  
# )

# # Train the MLP model
# nn_model_3_features.fit(X_train_3, y_train_3)

# # Evaluate on training data
# train_preds_3 = nn_model_3_features.predict(X_train_3)
# accuracy_train_3 = accuracy_score(y_train_3, train_preds_3)
# print(f"Training Accuracy with 3 features: {accuracy_train_3}")

# # Evaluate on validation data
# val_preds_3 = nn_model_3_features.predict(X_val_3)
# accuracy_val_3 = accuracy_score(y_val_3, val_preds_3)
# f1_val_3 = f1_score(y_val_3, val_preds_3, average='weighted')
# print(f"Validation Accuracy with 3 features: {accuracy_val_3}")
# print(f"Validation F1 Score with 3 features: {f1_val_3}")

# # Evaluate precision on validation data
# precision_val_3 = precision_score(y_val_3, val_preds_3, average='weighted')
# print(f"Validation Precision Score with 3 features: {precision_val_3}")

# # Save model with 3 features
# joblib.dump(nn_model_3_features, 'nn_model_3_features_smote.pkl')


In [ ]:
# %%
# # Train model with 9 features with more iterations
# nn_model_9_features = MLPClassifier(
#     hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
#     activation='relu',
#     solver='adam',
#     max_iter=300  
# )

# # Train the MLP model
# nn_model_9_features.fit(X_train_9, y_train_9)

# # Evaluate on training data
# train_preds_9 = nn_model_9_features.predict(X_train_9)
# accuracy_train_9 = accuracy_score(y_train_9, train_preds_9)
# print(f"Training Accuracy with 9 features: {accuracy_train_9}")

# # Evaluate on validation data
# val_preds_9 = nn_model_9_features.predict(X_val_9)
# accuracy_val_9 = accuracy_score(y_val_9, val_preds_9)
# f1_val_9 = f1_score(y_val_9, val_preds_9, average='weighted')

# print(f"Validation Accuracy with 9 features: {accuracy_val_9}")
# print(f"Validation F1 Score with 9 features: {f1_val_9}")

# # Evaluate precision on validation data
# precision_val_9 = precision_score(y_val_9, val_preds_9, average='weighted')
# print(f"Validation Precision Score with 9 features: {precision_val_9}")

# # Save model with 9 features
# joblib.dump(nn_model_9_features, 'nn_model_9_features_smote.pkl')


In [ ]:
# %%
# # Train model with improved features with more iterations
# nn_model_improved_features = MLPClassifier(
#     hidden_layer_sizes=(32, 16, 16, 8, 8, 4), 
#     activation='relu',
#     solver='adam',
#     max_iter=300  
# )

# # Train the MLP model
# nn_model_improved_features.fit(X_train_improved, y_train_improved)

# # Evaluate on training data
# train_preds_improved = nn_model_improved_features.predict(X_train_improved)

# accuracy_train_improved = accuracy_score(y_train_improved, train_preds_improved)
# print(f"Training Accuracy with improved features: {accuracy_train_improved}")

# # Evaluate on validation data
# val_preds_improved = nn_model_improved_features.predict(X_val_improved)
# accuracy_val_improved = accuracy_score(y_val_improved, val_preds_improved)
# f1_val_improved = f1_score(y_val_improved, val_preds_improved, average='weighted')

# print(f"Validation Accuracy with improved features: {accuracy_val_improved}")
# print(f"Validation F1 Score with improved features: {f1_val_improved}")

# # Evaluate precision on validation data
# precision_val_improved = precision_score(y_val_improved, val_preds_improved, average='weighted')
# print(f"Validation Precision Score with improved features: {precision_val_improved}")

# # Save model with improved features
# joblib.dump(nn_model_improved_features, 'nn_model_improved_features_smote.pkl')


In [ ]:
# %%
# # Get predictions for the test data with all models
# test_preds_all = nn_model_all_features.predict(test_data_NN_all)
# test_preds_3 = nn_model_3_features.predict(test_data_NN_3)
# test_preds_9 = nn_model_9_features.predict(test_data_NN_9)
# test_preds_improved = nn_model_improved_features.predict(test_data_NN_improved)


# # See the unique values in the predictions
# unique_values_all = np.unique(test_preds_all)
# print(unique_values_all)

# unique_values_3 = np.unique(test_preds_3)
# print(unique_values_3)

# unique_values_9 = np.unique(test_preds_9)
# print(unique_values_9)

# unique_values_improved = np.unique(test_preds_improved)
# print(unique_values_improved)

# # Format the predictions according to the required label format
# formatted_predictions_all = [claim_injury_type_mapping[pred] for pred in test_preds_all]
# formatted_predictions_3 = [claim_injury_type_mapping[pred] for pred in test_preds_3]
# formatted_predictions_9 = [claim_injury_type_mapping[pred] for pred in test_preds_9]
# formatted_predictions_improved = [claim_injury_type_mapping[pred] for pred in test_preds_improved]

# # Create the submission DataFrames
# submission_all = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions_all
# })

# submission_3 = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions_3
# })

# submission_9 = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions_9
# })

# submission_improved = pd.DataFrame({
#     "Claim Identifier": test_data.index,
#     "Claim Injury Type": formatted_predictions_improved
# })

# # Save to CSV in the required format
# submission_all.to_csv("Group39__Version05.csv", index=False)
# submission_3.to_csv("Group39__Version06.csv", index=False)
# submission_9.to_csv("Group39__Version07.csv", index=False)
# submission_improved.to_csv("Group39__Version08.csv", index=False)


# # Check length of the submission files
# print(len(submission_all))
# print(len(submission_3))
# print(len(submission_9))
# print(len(submission_improved))


In [ ]:
# %%
# #See the unique values in the submission files
# print(submission_all['Claim Injury Type'].value_counts())
# print(submission_3['Claim Injury Type'].value_counts())
# print(submission_9['Claim Injury Type'].value_counts())
# print(submission_improved['Claim Injury Type'].value_counts())


 ## (POOR RESULTS AND CODE COMMENTED OUT)Results with Smote and RFE

 * 3 features

     * Accuracy val data: 0.591

     * f1-score val data: 0.547

     * f1-score kaggle: 0.00...

     * NOTE: ONLY MINORITY CLASSES IN PREDICTIONS!! We don't trust this set of features at all to be good predicctors for target **on their own**

 * 9 features

     * Accuracy val data: 0.656

     * f1-score val data: 0.612

     * f1-score kaggle: 0.15

 * All features

     * Accuracy val data: 0.670

     * f1-score val data: 0.629

     * f1-score kaggle: 0.13

 * 10 ("improved") features

     * Accuracy val data: 0.665

     * f1-score val data: 0.623

     * f1-score kaggle:





 ## Addition of Average Weekly Wage back to model



 * Team member (and befoe this line also we) found out that this makes the f1-score better!

 ## Further NN Model Experiment: Change RFE to Decision Tree RFE instead of Logistic Regression RFE

In [ ]:
# %%
#TODO:


 ## SMOTE/Oversampling tried again.

 * Last time (also last time we tried, we didn't have wage column and also we potentially had leaky data preprocessing process) we tried SMOTE, it wasn't successcull. After less leaky data preprocessing, experimenting with different RFE methods and hyperparameter tuning, lets

 * We still have minority class members underpresneted

In [ ]:
# %%



 ## NN Model: Hyperparameter Tuning and other experiments

 * Try Robust scaler (from ML practicals: Neural Networks) for data instead of standard scaler

 * Hyperaparameter Tuning

     * Layer tuning

     * Solver tuning

     * Max iteration tuning

     * Other...

In [ ]:
# %%



 ## NN Model: Assessment

 * See precision, recall, f1-score, training and testing accuracy and confusion matrix and assess the quality of the model.